In [3]:
#import necessary libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [4]:
#create array for words, classes and documents and import intents file
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data = open('data/intents.json').read()
intents = json.loads(data)


## Data Preprocessing

In [5]:
#tokenize each sentence and create a list of classes for the tags
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [6]:
# lemmatization and data cleaning
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

259 documents
70 classes ['Consultation', 'Depression symptoms', 'Mental_Health', 'about', 'angry', 'anxiety', 'anxiety_causes', 'anxiety_disorders', 'anxiety_symptoms', 'bad', 'bipolar_causes', 'bipolar_disorder', 'bipolar_symptoms', 'bipolar_treatment', 'causes', 'definition', 'depression', 'depression prevention', 'do', 'eating_causes', 'eating_disorder', 'eating_symptoms', 'eating_treatment', 'fav', 'find_help', 'funny', 'generalized_disorder', 'good', 'goodbye', 'greetings', 'help', 'initiate', 'joke', 'life', 'name', 'name1', 'noanswer', 'obsessive_disorder', 'okay', 'panic_attacks', 'panic_disorder', 'personality_causes', 'personality_disorders', 'personality_symptoms', 'personality_treatment', 'phobias', 'post_disorder', 'post_symptoms', 'professional_types', 'recover', 'relationship', 'relax', 'right_professional', 'sad', 'scary_things', 'schizophrenia', 'situations', 'social', 'social_phobia', 'specific_phobia', 'steps', 'talk', 'tasks', 'thanks', 'tips', 'treat', 'treatement

In [7]:
# create training data array
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [8]:
# shuffle features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test data wherein X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])


<ipython-input-8-e1694c840fc4>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [15]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model using Stochastic gradient descent with Nesterov accelerated gradient
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model', hist)
print("model created")


C:\Users\lois\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
52/52 [==============================] - 1s 2ms/step - loss: 4.2615 - accuracy: 0.0154
Epoch 2/200
52/52 [==============================] - 0s 2ms/step - loss: 4.2073 - accuracy: 0.0386
Epoch 3/200
52/52 [==============================] - 0s 2ms/step - loss: 4.1215 - accuracy: 0.0463
Epoch 4/200
52/52 [==============================] - 0s 4ms/step - loss: 4.0564 - accuracy: 0.0811: 0s - loss: 4.0839 - accuracy: 
Epoch 5/200
52/52 [==============================] - 0s 2ms/step - loss: 3.9773 - accuracy: 0.0695
Epoch 6/200
52/52 [==============================] - 0s 2ms/step - loss: 3.8505 - accuracy: 0.1042
Epoch 7/200
52/52 [==============================] - 0s 2ms/step - loss: 3.7057 - accuracy: 0.1197
Epoch 8/200
52/52 [==============================] - 0s 2ms/step - loss: 3.5544 - accuracy: 0.1506
Epoch 9/200
52/52 [==============================] - 0s 2ms/step - loss: 3.3259 - accuracy: 0.1815
Epoch 10/200
52/52 [==============================] - 0s 2ms/step - loss: 3.0